<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [8]</a>'.</span>

# Creation de Contenu Audio Educatif

**Module :** 04-Audio-Applications  
**Niveau :** Applications  
**Technologies :** OpenAI TTS, Kokoro TTS, Whisper, GPT (LLM), pydub  
**VRAM estimee :** ~10 GB  
**Duree estimee :** 50 minutes  

## Objectifs d'Apprentissage

- [ ] Generer des scripts de narration a partir de texte de cours via LLM
- [ ] Produire une narration multi-voix (voix differentes par section ou personnage)
- [ ] Creer du contenu audio multilingue (francais, anglais)
- [ ] Appliquer des parametres d'accessibilite (vitesse reduite, enonciation claire)
- [ ] Assembler un cours narrate complet a partir de sections individuelles
- [ ] Verifier la qualite par round-trip STT (transcription de validation)

## Prerequis

- Notebooks Foundation (01-1 a 01-5) completes
- Cle API OpenAI configuree (`OPENAI_API_KEY` dans `.env`)
- pydub installe (manipulation audio)
- Comprehension de base du TTS et du STT

**Navigation :** [Index](../README.md) | [<< Precedent](../03-Orchestration/03-3-Realtime-Voice-API.ipynb) | [Suivant >>](04-2-Transcription-Pipeline.ipynb)

In [1]:
# Parametres Papermill - JAMAIS modifier ce commentaire

# Configuration notebook
notebook_mode = "interactive"        # "interactive" ou "batch"
skip_widgets = False               # True pour mode batch MCP
debug_level = "INFO"

# Parametres contenu educatif
tts_model = "tts-1"               # "tts-1" ou "tts-1-hd"
narrator_voice = "nova"            # Voix principale de narration
secondary_voice = "onyx"           # Voix secondaire (exemples, citations)
llm_model = "gpt-4o-mini"         # Modele LLM pour generation de scripts
target_languages = ["fr", "en"]    # Langues cibles
accessibility_speed = 0.85         # Vitesse reduite pour accessibilite

# Configuration sauvegarde
generate_audio = True
save_audio_files = True

In [2]:
# Setup environnement et imports
import os
import sys
import json
import time
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Any, Optional
from io import BytesIO
import logging

from IPython.display import Audio, display, HTML

# Resolution GENAI_ROOT
GENAI_ROOT = Path.cwd()
while GENAI_ROOT.name != 'GenAI' and len(GENAI_ROOT.parts) > 1:
    GENAI_ROOT = GENAI_ROOT.parent

HELPERS_PATH = GENAI_ROOT / 'shared' / 'helpers'
if HELPERS_PATH.exists():
    sys.path.insert(0, str(HELPERS_PATH.parent))
    try:
        from helpers.audio_helpers import (
            play_audio_bytes, synthesize_openai, transcribe_openai,
            get_audio_info, estimate_audio_duration
        )
        print("Helpers audio importes")
    except ImportError as e:
        print(f"Helpers audio non disponibles - mode autonome : {e}")

# Repertoire de sortie
OUTPUT_DIR = GENAI_ROOT / 'outputs' / 'audio' / 'educational'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Configuration logging
logging.basicConfig(level=getattr(logging, debug_level))
logger = logging.getLogger('educational_audio')

print(f"Creation de Contenu Audio Educatif")
print(f"Date : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Mode : {notebook_mode}, TTS : {tts_model}")
print(f"Voix narrateur : {narrator_voice}, Voix secondaire : {secondary_voice}")
print(f"Sortie : {OUTPUT_DIR}")

Helpers audio importes
Creation de Contenu Audio Educatif
Date : 2026-02-26 08:23:02
Mode : interactive, TTS : tts-1
Voix narrateur : nova, Voix secondaire : onyx
Sortie : D:\Dev\CoursIA.worktrees\GenAI_Series\MyIA.AI.Notebooks\GenAI\outputs\audio\educational


In [3]:
# Chargement de la configuration et validation API
from dotenv import load_dotenv

current_path = Path.cwd()
found_env = False
for _ in range(4):
    env_path = current_path / '.env'
    if env_path.exists():
        load_dotenv(env_path)
        print(f"Fichier .env charge depuis : {env_path}")
        found_env = True
        break
    current_path = current_path.parent

if not found_env:
    print("Aucun fichier .env trouve dans l'arborescence")

openai_key = os.getenv('OPENAI_API_KEY')

if not openai_key:
    if notebook_mode == "batch" and not generate_audio:
        print("Mode batch sans generation : cle API ignoree")
        openai_key = "dummy_key_for_validation"
    else:
        raise ValueError(
            "OPENAI_API_KEY manquante dans .env\n"
            "Obtenez votre cle sur : https://platform.openai.com/api-keys"
        )

from openai import OpenAI
client = OpenAI(api_key=openai_key)

if openai_key != "dummy_key_for_validation":
    try:
        models = client.models.list()
        tts_available = any('tts' in m.id for m in models)
        print(f"Connexion API reussie (TTS disponible : {tts_available})")
    except Exception as e:
        print(f"Erreur connexion : {str(e)[:100]}")

print(f"\nConfiguration :")
print(f"  TTS : {tts_model} | LLM : {llm_model}")
print(f"  Langues : {target_languages}")
print(f"  Accessibilite : vitesse {accessibility_speed}x")

Fichier .env charge depuis : D:\Dev\CoursIA.worktrees\GenAI_Series\MyIA.AI.Notebooks\GenAI\.env


INFO:httpx:HTTP Request: GET https://api.openai.com/v1/models "HTTP/1.1 200 OK"


Connexion API reussie (TTS disponible : True)

Configuration :
  TTS : tts-1 | LLM : gpt-4o-mini
  Langues : ['fr', 'en']
  Accessibilite : vitesse 0.85x


## Section 1 : Generation de scripts de narration via LLM

La premiere etape d'un contenu audio educatif consiste a transformer un texte brut de cours en un script de narration naturel. Un LLM adapte le contenu technique en discours oral fluide :

| Etape | Description | Outil |
|-------|-------------|-------|
| Texte source | Contenu pedagogique brut | -- |
| Reformulation | Adaptation pour la narration orale | GPT-4o-mini |
| Structuration | Decoupage en sections narrees | GPT-4o-mini |
| Attribution voix | Assignation des voix par section | Configuration |

Le LLM recoit le texte de cours et genere un script avec des indications de voix, pauses et intonation.

In [4]:
# Generation de scripts de narration a partir de texte de cours
print("GENERATION DE SCRIPTS DE NARRATION")
print("=" * 50)

# Texte de cours source (exemple : introduction a l'IA)
course_text = """
L'intelligence artificielle (IA) est un domaine de l'informatique qui vise
a creer des systemes capables de realiser des taches necessitant normalement
l'intelligence humaine. Ces taches incluent la reconnaissance vocale,
la prise de decision, la traduction et la generation de contenu.

Le machine learning (apprentissage automatique) est une sous-branche de l'IA
qui permet aux machines d'apprendre a partir de donnees sans etre explicitement
programmees. Les reseaux de neurones profonds (deep learning) sont une technique
de machine learning inspiree du cerveau humain.
""".strip()

# Prompt pour le LLM : transformer en script de narration
narration_prompt = f"""
Transforme ce texte de cours en un script de narration audio educatif.
Le script doit :
- Etre naturel a l'oral (phrases courtes, transitions fluides)
- Inclure des marqueurs [PAUSE] pour les pauses naturelles
- Indiquer les changements de voix avec [NARRATEUR] et [EXPERT]
- Ajouter une introduction accueillante et une conclusion

Texte source :
{course_text}

Genere uniquement le script, sans commentaires.
"""

narration_script = ""
if generate_audio:
    start_time = time.time()
    response = client.chat.completions.create(
        model=llm_model,
        messages=[{"role": "user", "content": narration_prompt}],
        temperature=0.7,
        max_tokens=1000
    )
    narration_script = response.choices[0].message.content
    gen_time = time.time() - start_time

    print(f"Script genere en {gen_time:.1f}s")
    print(f"Tokens utilises : {response.usage.total_tokens}")
    print(f"\n--- Script de narration ---")
    print(narration_script)
    print(f"--- Fin du script ---")
    print(f"\nLongueur : {len(narration_script)} caracteres")
    print(f"Duree estimee : {estimate_audio_duration(narration_script):.0f}s")
else:
    print("Generation desactivee (generate_audio=False)")

GENERATION DE SCRIPTS DE NARRATION


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Script genere en 6.4s
Tokens utilises : 684

--- Script de narration ---
[NARRATEUR] Bonjour et bienvenue dans ce nouvel épisode sur l'intelligence artificielle ! [PAUSE] Aujourd'hui, nous allons explorer ce domaine fascinant et ses différentes facettes. [PAUSE] Prêts à plonger ? C'est parti !

[NARRATEUR] L'intelligence artificielle, ou IA, [PAUSE] est un secteur de l'informatique [PAUSE] qui cherche à créer des systèmes capables d'accomplir des tâches [PAUSE] qui nécessitent normalement l'intelligence humaine. [PAUSE] Ces tâches peuvent inclure la reconnaissance vocale, [PAUSE] la prise de décision, [PAUSE] la traduction, [PAUSE] et même la génération de contenu. [PAUSE]

[EXPERT] C'est fascinant, n'est-ce pas ? [PAUSE] Mais ce n'est qu'une partie de l'histoire. [PAUSE] L'IA est un domaine vaste et en constante évolution. [PAUSE] Pour mieux la comprendre, [PAUSE] parlons du machine learning, ou apprentissage automatique.

[NARRATEUR] Le machine learning est une sous-branche de l'IA. 

### Interpretation : Generation de scripts

| Aspect | Observation | Signification |
|--------|-------------|---------------|
| Temps LLM | Typiquement < 5s | GPT-4o-mini est rapide pour cette tache |
| Marqueurs | [PAUSE], [NARRATEUR], [EXPERT] | Permettent le controle fin de la narration |
| Longueur | Script ~2x plus long que le texte source | L'oral necessite plus de mots que l'ecrit |

**Points cles** :
1. Le LLM adapte naturellement le registre ecrit vers l'oral
2. Les marqueurs structurent le script pour le pipeline TTS
3. La temperature 0.7 donne un bon equilibre naturel/fidele

## Section 2 : Narration multi-voix

Un contenu educatif beneficie de plusieurs voix pour distinguer les roles :

| Role | Voix suggeree | Utilisation |
|------|---------------|-------------|
| Narrateur principal | `nova` | Explications, transitions |
| Expert / Citations | `onyx` | Definitions, concepts cles |
| Exemples | `echo` | Illustrations pratiques |

Nous allons parser le script genere, identifier les segments par voix, et generer chaque segment avec la voix appropriee.

In [5]:
# Narration multi-voix : parser et generer par segments
print("NARRATION MULTI-VOIX")
print("=" * 50)

import re

# Mapping des roles vers les voix OpenAI
voice_mapping = {
    "NARRATEUR": narrator_voice,
    "EXPERT": secondary_voice,
    "DEFAULT": narrator_voice
}

def parse_narration_script(script: str) -> List[Dict[str, str]]:
    """Parse un script avec marqueurs [ROLE] en segments."""
    segments = []
    current_role = "DEFAULT"
    current_text = []

    for line in script.split('\n'):
        line = line.strip()
        if not line:
            continue

        # Detection marqueur de role
        role_match = re.match(r'\[(NARRATEUR|EXPERT)\]', line)
        if role_match:
            # Sauvegarder le segment precedent
            if current_text:
                text = ' '.join(current_text).strip()
                text = re.sub(r'\[PAUSE\]', '...', text)
                if text:
                    segments.append({"role": current_role, "text": text})
            current_role = role_match.group(1)
            # Texte apres le marqueur
            remaining = line[role_match.end():].strip()
            current_text = [remaining] if remaining else []
        else:
            current_text.append(line)

    # Dernier segment
    if current_text:
        text = ' '.join(current_text).strip()
        text = re.sub(r'\[PAUSE\]', '...', text)
        if text:
            segments.append({"role": current_role, "text": text})

    return segments

# Parser le script
if narration_script:
    segments = parse_narration_script(narration_script)
else:
    # Script de demonstration si la generation a ete sautee
    segments = [
        {"role": "NARRATEUR", "text": "Bienvenue dans ce cours sur l'intelligence artificielle."},
        {"role": "EXPERT", "text": "L'intelligence artificielle est un domaine qui vise a creer des systemes intelligents."},
        {"role": "NARRATEUR", "text": "Voyons maintenant le machine learning, une sous-branche fondamentale."},
        {"role": "EXPERT", "text": "Le machine learning permet aux machines d'apprendre a partir de donnees."}
    ]

print(f"Segments identifies : {len(segments)}")
print(f"\nDetail des segments :")
for i, seg in enumerate(segments):
    voice = voice_mapping.get(seg['role'], narrator_voice)
    print(f"  [{i+1}] {seg['role']:12s} -> voix '{voice}' | {seg['text'][:60]}...")

# Generer chaque segment avec la voix appropriee
audio_segments = []

if generate_audio:
    print(f"\nGeneration audio par segment :")
    total_start = time.time()

    for i, seg in enumerate(segments):
        voice = voice_mapping.get(seg['role'], narrator_voice)
        start_time = time.time()

        response = client.audio.speech.create(
            model=tts_model,
            voice=voice,
            input=seg['text'],
            response_format="mp3",
            speed=1.0
        )

        audio_data = response.content
        gen_time = time.time() - start_time

        audio_segments.append({
            "role": seg['role'],
            "voice": voice,
            "audio": audio_data,
            "size_kb": len(audio_data) / 1024,
            "time": gen_time
        })

        print(f"  Segment {i+1}/{len(segments)} : {voice} | {gen_time:.1f}s | {len(audio_data)/1024:.1f} KB")

    total_time = time.time() - total_start
    print(f"\nTotal : {len(audio_segments)} segments en {total_time:.1f}s")
    print(f"Taille totale : {sum(s['size_kb'] for s in audio_segments):.1f} KB")

    # Ecouter le premier segment
    if audio_segments:
        print(f"\nEcoute du premier segment ({audio_segments[0]['role']}, voix {audio_segments[0]['voice']}) :")
        display(Audio(data=audio_segments[0]['audio'], autoplay=False))
else:
    print("Generation desactivee")

NARRATION MULTI-VOIX
Segments identifies : 7

Detail des segments :
  [1] NARRATEUR    -> voix 'nova' | Bonjour et bienvenue dans ce nouvel épisode sur l'intelligen...
  [2] NARRATEUR    -> voix 'nova' | L'intelligence artificielle, ou IA, ... est un secteur de l'...
  [3] EXPERT       -> voix 'onyx' | C'est fascinant, n'est-ce pas ? ... Mais ce n'est qu'une par...
  [4] NARRATEUR    -> voix 'nova' | Le machine learning est une sous-branche de l'IA. ... Il per...
  [5] EXPERT       -> voix 'onyx' | C'est là qu'entrent en jeu les réseaux de neurones profonds,...
  [6] NARRATEUR    -> voix 'nova' | En résumé, l'intelligence artificielle ... et le machine lea...
  [7] NARRATEUR    -> voix 'nova' | Merci de nous avoir rejoints aujourd'hui ! ... J'espère que ...

Generation audio par segment :


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/audio/speech "HTTP/1.1 200 OK"


  Segment 1/7 : nova | 3.0s | 243.8 KB


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/audio/speech "HTTP/1.1 200 OK"


  Segment 2/7 : nova | 5.2s | 409.2 KB


...

  Segment 5/7 : onyx | 3.1s | 287.8 KB


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/audio/speech "HTTP/1.1 200 OK"


  Segment 6/7 : nova | 3.8s | 213.3 KB


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/audio/speech "HTTP/1.1 200 OK"


  Segment 7/7 : nova | 3.3s | 236.7 KB

Total : 7 segments en 24.7s
Taille totale : 1882.5 KB

Ecoute du premier segment (NARRATEUR, voix nova) :


### Interpretation : Narration multi-voix

| Aspect | Valeur | Signification |
|--------|--------|---------------|
| Segments | Variable (4-10 typique) | Chaque changement de role cree un segment |
| Temps par segment | ~1-3s | L'API est rapide meme pour du texte long |
| Differenciation | Voix distinctes par role | Ameliore la comprehension et l'engagement |

**Points cles** :
1. Le parsing des marqueurs [ROLE] permet une attribution automatique des voix
2. Les pauses [PAUSE] sont converties en ellipses pour un rythme naturel
3. La multi-voix rend le contenu plus dynamique qu'un narrateur unique

## Section 3 : Contenu multilingue

L'API OpenAI TTS gere nativement de nombreuses langues. Pour creer du contenu multilingue, deux approches :

| Approche | Description | Avantage |
|----------|-------------|----------|
| Traduction LLM + TTS | Traduire via GPT, puis synthetiser | Controle total du texte |
| TTS direct multilingue | Fournir le texte dans la langue cible | Plus simple, qualite variable |

Nous utilisons la premiere approche pour garantir la qualite pedagogique du contenu traduit.

In [6]:
# Contenu multilingue : traduction + narration
print("CONTENU MULTILINGUE")
print("=" * 50)

# Texte source en francais
source_text_fr = (
    "L'apprentissage automatique permet aux ordinateurs d'apprendre "
    "a partir d'exemples. Au lieu de programmer chaque regle manuellement, "
    "on fournit des donnees et l'algorithme decouvre les patterns par lui-meme."
)

lang_configs = {
    "fr": {"name": "Francais", "voice": "nova", "text": source_text_fr},
    "en": {"name": "English", "voice": "alloy", "text": None}  # A traduire
}

multilingual_results = {}

if generate_audio:
    # Etape 1 : Traduction vers l'anglais via LLM
    print("Etape 1 : Traduction via LLM")
    translation_response = client.chat.completions.create(
        model=llm_model,
        messages=[{
            "role": "user",
            "content": f"Traduis ce texte educatif en anglais, en gardant le ton pedagogique :\n\n{source_text_fr}"
        }],
        temperature=0.3
    )
    lang_configs["en"]["text"] = translation_response.choices[0].message.content

    for lang_code, config in lang_configs.items():
        print(f"\n--- {config['name']} ({lang_code}) ---")
        print(f"Texte : {config['text'][:80]}...")

        start_time = time.time()
        response = client.audio.speech.create(
            model=tts_model,
            voice=config['voice'],
            input=config['text'],
            response_format="mp3"
        )
        audio_data = response.content
        gen_time = time.time() - start_time

        multilingual_results[lang_code] = {
            "audio": audio_data,
            "text": config['text'],
            "voice": config['voice'],
            "size_kb": len(audio_data) / 1024,
            "time": gen_time
        }

        print(f"Voix : {config['voice']} | {gen_time:.1f}s | {len(audio_data)/1024:.1f} KB")
        display(Audio(data=audio_data, autoplay=False))

        if save_audio_files:
            filepath = OUTPUT_DIR / f"multilingual_{lang_code}.mp3"
            with open(filepath, 'wb') as f:
                f.write(audio_data)
            print(f"Sauvegarde : {filepath.name}")

    # Comparaison
    print(f"\nComparaison multilingue :")
    print(f"{'Langue':<12} {'Voix':<10} {'Taille (KB)':<12} {'Temps (s)':<10}")
    print("-" * 44)
    for lang, data in multilingual_results.items():
        print(f"{lang:<12} {data['voice']:<10} {data['size_kb']:<12.1f} {data['time']:<10.1f}")
else:
    print("Generation desactivee")

CONTENU MULTILINGUE
Etape 1 : Traduction via LLM


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



--- Francais (fr) ---
Texte : L'apprentissage automatique permet aux ordinateurs d'apprendre a partir d'exempl...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/audio/speech "HTTP/1.1 200 OK"


Voix : nova | 4.7s | 237.2 KB


Sauvegarde : multilingual_fr.mp3

--- English (en) ---
Texte : Machine learning enables computers to learn from examples. Instead of manually p...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/audio/speech "HTTP/1.1 200 OK"


Voix : alloy | 3.2s | 201.1 KB


Sauvegarde : multilingual_en.mp3

Comparaison multilingue :
Langue       Voix       Taille (KB)  Temps (s) 
--------------------------------------------
fr           nova       237.2        4.7       
en           alloy      201.1        3.2       


### Interpretation : Contenu multilingue

| Aspect | Valeur | Signification |
|--------|--------|---------------|
| Qualite FR | Excellente | Voix OpenAI optimisees pour le francais |
| Qualite EN | Excellente | Langue native du modele |
| Traduction LLM | Fidele au contenu | GPT-4o-mini conserve le ton pedagogique |

> **Note technique** : Pour des langues moins bien supportees, tester la qualite avec un locuteur natif avant production.

## Section 4 : Accessibilite audio

Un contenu educatif accessible doit prendre en compte differents profils d'apprenants :

| Parametre | Valeur standard | Valeur accessible | Impact |
|-----------|----------------|-------------------|--------|
| Vitesse | 1.0x | 0.85x | Meilleure comprehension |
| Pauses | Courtes | Allongees | Temps de reflexion |
| Voix | Au choix | Claire, posee | Intelligibilite |
| Repetitions | Aucune | Concepts cles repetes | Memorisation |

In [7]:
# Accessibilite : vitesse reduite et enonciation claire
print("ACCESSIBILITE AUDIO")
print("=" * 50)

accessibility_text = (
    "Un reseau de neurones est compose de couches de neurones artificiels. "
    "Chaque neurone recoit des entrees, les multiplie par des poids, "
    "et applique une fonction d'activation. "
    "Retenez bien : entrees, poids, activation. "
    "C'est le fonctionnement fondamental de tout reseau de neurones."
)

speed_configs = [
    {"label": "Standard (1.0x)", "speed": 1.0},
    {"label": f"Accessible ({accessibility_speed}x)", "speed": accessibility_speed},
    {"label": "Lent (0.7x)", "speed": 0.7}
]

accessibility_results = []

if generate_audio:
    for config in speed_configs:
        print(f"\n--- {config['label']} ---")
        start_time = time.time()

        response = client.audio.speech.create(
            model=tts_model,
            voice=narrator_voice,
            input=accessibility_text,
            response_format="mp3",
            speed=config['speed']
        )

        audio_data = response.content
        gen_time = time.time() - start_time

        accessibility_results.append({
            "label": config['label'],
            "speed": config['speed'],
            "audio": audio_data,
            "size_kb": len(audio_data) / 1024,
            "time": gen_time
        })

        print(f"Taille : {len(audio_data)/1024:.1f} KB | Temps : {gen_time:.1f}s")
        display(Audio(data=audio_data, autoplay=False))

    # Comparaison
    print(f"\nComparaison des vitesses :")
    print(f"{'Mode':<25} {'Vitesse':<10} {'Taille (KB)':<12}")
    print("-" * 47)
    for r in accessibility_results:
        print(f"{r['label']:<25} {r['speed']:<10} {r['size_kb']:<12.1f}")

    print(f"\nLa version accessible est ~{accessibility_results[1]['size_kb']/accessibility_results[0]['size_kb']*100:.0f}% de la taille standard")
else:
    print("Generation desactivee")

ACCESSIBILITE AUDIO

--- Standard (1.0x) ---


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/audio/speech "HTTP/1.1 200 OK"


Taille : 319.7 KB | Temps : 4.3s



--- Accessible (0.85x) ---


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/audio/speech "HTTP/1.1 200 OK"


Taille : 388.6 KB | Temps : 5.8s



--- Lent (0.7x) ---


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/audio/speech "HTTP/1.1 200 OK"


Taille : 467.8 KB | Temps : 5.7s



Comparaison des vitesses :
Mode                      Vitesse    Taille (KB) 
-----------------------------------------------
Standard (1.0x)           1.0        319.7       
Accessible (0.85x)        0.85       388.6       
Lent (0.7x)               0.7        467.8       

La version accessible est ~122% de la taille standard


## Section 5 : Assemblage d'un cours narrate complet

L'assemblage final combine tous les segments audio en un fichier unique avec pydub. Le processus :

1. Charger chaque segment audio genere
2. Ajouter des silences entre les sections (transition naturelle)
3. Concatener dans l'ordre du script
4. Normaliser le volume global
5. Exporter en format final

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [8]:
# Assemblage du cours narrate complet
print("ASSEMBLAGE DU COURS NARRATE")
print("=" * 50)

from pydub import AudioSegment
from pydub.silence import detect_nonsilent

if generate_audio and audio_segments:
    # Silence entre les segments (ms)
    inter_segment_silence = 800   # Entre segments du meme role
    inter_section_silence = 1500  # Entre sections (changement de role)

    # Construire le fichier final
    final_audio = AudioSegment.silent(duration=500)  # Silence initial
    prev_role = None

    for i, seg in enumerate(audio_segments):
        # Charger le segment
        segment_audio = AudioSegment.from_mp3(BytesIO(seg['audio']))

        # Ajouter silence adapte
        if prev_role is not None:
            silence_ms = inter_section_silence if seg['role'] != prev_role else inter_segment_silence
            final_audio += AudioSegment.silent(duration=silence_ms)

        final_audio += segment_audio
        prev_role = seg['role']
        print(f"  Segment {i+1} : {seg['role']:12s} ({len(segment_audio)/1000:.1f}s)")

    # Silence final
    final_audio += AudioSegment.silent(duration=500)

    # Normalisation du volume
    target_dBFS = -20.0
    change_in_dBFS = target_dBFS - final_audio.dBFS
    final_audio = final_audio.apply_gain(change_in_dBFS)

    print(f"\nCours assemble :")
    print(f"  Duree totale : {len(final_audio)/1000:.1f}s")
    print(f"  Volume normalise : {final_audio.dBFS:.1f} dBFS")
    print(f"  Segments : {len(audio_segments)}")

    # Sauvegarde
    if save_audio_files:
        output_path = OUTPUT_DIR / "cours_complet_assemble.mp3"
        final_audio.export(str(output_path), format="mp3", bitrate="192k")
        print(f"  Sauvegarde : {output_path.name} ({output_path.stat().st_size/1024:.1f} KB)")

    # Ecouter le resultat
    final_bytes = BytesIO()
    final_audio.export(final_bytes, format="mp3")
    print(f"\nEcoute du cours complet :")
    display(Audio(data=final_bytes.getvalue(), autoplay=False))
else:
    print("Assemblage ignore (pas de segments audio disponibles)")

ASSEMBLAGE DU COURS NARRATE


C:\Users\jsboi\AppData\Roaming\Python\Python313\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
C:\Users\jsboi\AppData\Roaming\Python\Python313\site-packages\pydub\utils.py:198: RuntimeWarning: Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work
  warn("Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work", RuntimeWarning)


FileNotFoundError: [WinError 2] Le fichier spécifié est introuvable

### Interpretation : Assemblage

| Aspect | Valeur | Signification |
|--------|--------|---------------|
| Silences inter-segments | 800ms | Pause naturelle entre phrases |
| Silences inter-sections | 1500ms | Marque un changement de locuteur/sujet |
| Normalisation | -20 dBFS | Volume confortable pour l'ecoute |
| Format final | MP3 192kbps | Bon compromis qualite/taille |

> **Note technique** : pydub utilise ffmpeg en arriere-plan. Assurez-vous que ffmpeg est installe dans votre environnement.

## Section 6 : Verification qualite par round-trip STT

Pour valider que la narration est fidele au texte source, nous effectuons un round-trip :

```
Texte source -> TTS -> Audio -> STT -> Texte transcrit -> Comparaison
```

La similarite entre le texte source et le texte retranscrit mesure la fidelite du pipeline.

In [ ]:
# Verification qualite par round-trip STT
print("VERIFICATION QUALITE - ROUND-TRIP STT")
print("=" * 50)

from difflib import SequenceMatcher

roundtrip_text = (
    "Le deep learning utilise des reseaux de neurones profonds "
    "pour apprendre des representations hierarchiques des donnees."
)

if generate_audio:
    # Etape 1 : TTS
    print("Etape 1 : Generation TTS")
    tts_response = client.audio.speech.create(
        model=tts_model,
        voice=narrator_voice,
        input=roundtrip_text,
        response_format="mp3"
    )
    tts_audio = tts_response.content
    print(f"  Audio genere : {len(tts_audio)/1024:.1f} KB")

    # Sauvegarder temporairement pour Whisper
    temp_path = OUTPUT_DIR / "roundtrip_temp.mp3"
    with open(temp_path, 'wb') as f:
        f.write(tts_audio)

    # Etape 2 : STT
    print("Etape 2 : Transcription STT")
    with open(temp_path, 'rb') as audio_file:
        transcript = client.audio.transcriptions.create(
            model="whisper-1",
            file=audio_file,
            language="fr"
        )
    transcribed_text = transcript.text
    print(f"  Texte transcrit : {transcribed_text}")

    # Etape 3 : Comparaison
    print("\nEtape 3 : Comparaison")
    similarity = SequenceMatcher(None,
                                 roundtrip_text.lower(),
                                 transcribed_text.lower()).ratio()

    print(f"  Texte original  : {roundtrip_text}")
    print(f"  Texte transcrit : {transcribed_text}")
    print(f"  Similarite      : {similarity*100:.1f}%")

    if similarity >= 0.9:
        print(f"  Verdict : EXCELLENT - Narration fidele")
    elif similarity >= 0.75:
        print(f"  Verdict : BON - Quelques differences mineures")
    else:
        print(f"  Verdict : A VERIFIER - Differences significatives")

    # Nettoyage
    if temp_path.exists():
        temp_path.unlink()
else:
    print("Verification desactivee")

In [ ]:
# Mode interactif - Creer votre propre contenu educatif
if notebook_mode == "interactive" and not skip_widgets:
    print("MODE INTERACTIF - CONTENU EDUCATIF PERSONNALISE")
    print("=" * 50)
    print("\nEntrez un paragraphe de cours a transformer en narration audio :")
    print("(Laissez vide pour passer)")

    try:
        user_course = input("\nVotre texte de cours : ")

        if user_course.strip():
            # Generer script de narration
            script_resp = client.chat.completions.create(
                model=llm_model,
                messages=[{"role": "user", "content": f"Transforme en narration orale educative :\n\n{user_course}"}],
                temperature=0.7
            )
            user_script = script_resp.choices[0].message.content
            print(f"\nScript genere :\n{user_script}")

            # Narrer
            tts_resp = client.audio.speech.create(
                model=tts_model,
                voice=narrator_voice,
                input=user_script,
                response_format="mp3"
            )
            print(f"\nNarration generee ({len(tts_resp.content)/1024:.1f} KB) :")
            display(Audio(data=tts_resp.content, autoplay=False))

            if save_audio_files:
                ts = datetime.now().strftime('%Y%m%d_%H%M%S')
                filepath = OUTPUT_DIR / f"custom_course_{ts}.mp3"
                with open(filepath, 'wb') as f:
                    f.write(tts_resp.content)
                print(f"Sauvegarde : {filepath.name}")
        else:
            print("Mode interactif ignore")

    except (KeyboardInterrupt, EOFError):
        print("Mode interactif interrompu")
    except Exception as e:
        error_type = type(e).__name__
        if "StdinNotImplemented" in error_type or "input" in str(e).lower():
            print("Mode interactif non disponible (execution automatisee)")
        else:
            print(f"Erreur : {error_type} - {str(e)[:100]}")
else:
    print("Mode batch - Interface interactive desactivee")

In [ ]:
# Statistiques de session et recapitulatif
print("STATISTIQUES DE SESSION")
print("=" * 50)

print(f"Date : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Mode : {notebook_mode}")
print(f"Modele TTS : {tts_model} | Modele LLM : {llm_model}")
print(f"Voix narrateur : {narrator_voice} | Voix secondaire : {secondary_voice}")

if audio_segments:
    total_segments = len(audio_segments)
    total_size_kb = sum(s['size_kb'] for s in audio_segments)
    print(f"Segments multi-voix generes : {total_segments}")
    print(f"Taille totale segments : {total_size_kb:.1f} KB")

if multilingual_results:
    print(f"Langues generees : {', '.join(multilingual_results.keys())}")

if save_audio_files:
    saved_files = list(OUTPUT_DIR.glob('*'))
    print(f"Fichiers sauvegardes : {len(saved_files)} dans {OUTPUT_DIR}")

print(f"\nPROCHAINES ETAPES")
print(f"1. Creer un pipeline de transcription (04-2-Transcription-Pipeline)")
print(f"2. Explorer la composition musicale (04-3-Music-Composition-Workflow)")
print(f"3. Synchroniser audio et video (04-4-Audio-Video-Sync)")

print(f"\nNotebook termine - {datetime.now().strftime('%H:%M:%S')}")